In [1]:
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [7]:
from scripts.detect_voids import get_neightbours, search_voids_bb_neightbours
from packages.yolo_predict import YOLO_Pred
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import cv2
import multiprocessing
from functools import partial

In [5]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [69]:
image = cv2.imread('/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

[autoreload of scripts.detect_voids failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "../scripts/detect_voids.py", line 1, in <module>
    from scripts.detect_voids import get_neightbours, search_voids_bb_neightbours
ImportError: cannot import name 'get_neightbours'
]


In [6]:
df_predictions = yolo.predictions(image)

In [13]:
def bb_intersection_over_union(df_predictions: pd.DataFrame, index_key:int, index_neightbour:int) -> float:

    a = index_key
    xA1 = df_predictions.loc[a][0] - df_predictions.loc[a][2]/2
    yA1 = df_predictions.loc[a][1] + df_predictions.loc[a][3]/2
    xA2 = df_predictions.loc[a][0] + df_predictions.loc[a][2]/2
    yA2 = df_predictions.loc[a][1] - df_predictions.loc[a][3]/2
    boxA = [xA1, yA1, xA2, yA2 ]

    b = index_neightbour # indices vecinos detectados que estan en el diccionario
    xB1 = df_predictions.loc[b][0] - df_predictions.loc[b][2]/2
    yB1 = df_predictions.loc[b][1] + df_predictions.loc[b][3]/2
    xB2 = df_predictions.loc[b][0] + df_predictions.loc[b][2]/2
    yB2 = df_predictions.loc[b][1] - df_predictions.loc[b][3]/2
    boxB = [xB1, yB1,xB2, yB2 ]
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])


    # compute the area of intersection rectangle
    interArea = (xB - xA) * (yB - yA)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
# para left w = -w y h = 0
# para right w = +w y h = 0
# para top w = 0 y h = + h

In [47]:
image = cv2.imread(filename='/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg')

In [18]:
image.shape[0:2]

(3264, 2448)

In [130]:
df_predictions.loc[1059]

Center_X    210.405991
Center_Y    233.077942
Width        24.668375
Height       70.583984
Name: 1059, dtype: float64

In [129]:
df_predictions.loc[1059][2]

24.66837501525879

In [123]:
xA1 = df_predictions.loc[1059][0] - df_predictions.loc[1059][2]/2 - w_index_a
yA1 = df_predictions.loc[1059][1] - df_predictions.loc[1059][3]/2
xA2 = df_predictions.loc[1059][0] + df_predictions.loc[1059][2]/2 - w_index_a
yA2 = df_predictions.loc[1059][1] + df_predictions.loc[1059][3]/2

In [125]:
xA1, yA1, xA2, yA2

(184.86670780181885,
 268.36993408203125,
 209.53508281707764,
 197.78594970703125)

In [103]:
def search_voids_bb_neightbours_left(df_predictions: pd.DataFrame, dict_of_neightbours: dict):
    
    list_of_voids = []
    for index_a, index_b in dict_of_neightbours.items():

        first_lists = []
        h_image, w_image = image.shape[0:2] # obtengo limites de la imagen
        w_index_a = df_predictions.loc[int(index_a)][2]
        h_index_a = df_predictions.loc[int(index_a)][3]
        
        xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 - w_index_a
        yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
        xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 - w_index_a
        yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2
        boxA = [xA1, yA1, xA2, yA2]

        void_number = 1
        void_text = f'Void number {void_number}'        
        X_center_A = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]  #X_center - Width
        Y_center_A = df_predictions.loc[int(index_a)][1]  - df_predictions.loc[int(index_a)][2]  #Y_center - Width

        print(f'Evaluating {index_a}...')
        if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
                trigger = True
                for item in index_b:
                    xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
                    yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
                    xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
                    yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
                    boxB = [xB1, yB1,xB2, yB2]

                    xA = max(boxA[0], boxB[0])
                    yA = max(boxA[1], boxB[1])
                    xB = min(boxA[2], boxB[2])
                    yB = min(boxA[3], boxB[3])

                    interArea = (xB - xA) * (yB - yA)

                    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
                    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

                    iou = interArea / float(boxAArea + boxBArea - interArea)
                    trigger = trigger and (iou < 0.1)
                #print(f'Trigger: {trigger}')
                if trigger == False:
                    pass
                    #print(f'No hay espacio vacio a la izquierda de {index_a}')
                else:
                    print(f'A la izquierda de {index_a} hay espacio vacio')
                    
                    list_of_voids.append(boxA)

    return list_of_voids

In [ ]:
# izquierda (-1)
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 - k * w_index_a
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 - k * w_index_a
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2

In [ ]:
# derecha (+1)
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + k * w_index_a
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + k * w_index_a
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2

In [ ]:
# arriba 
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 - h_index_a
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 - h_index_a

In [ ]:
# arriba 
xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + k * w_index_a # Para izquierda: (-1) / Para derecha: (1) / Para arriba: 0
yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 + k * h_index_a # Para izquierda: 0 / Para derecha: 0 / Para arriba: -1
xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + k * w_index_a # Para izquierda: (-1) / Para derecha: (1) / Para arriba: 0
yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 + K * h_index_a # Para izquierda: 0 / Para derecha: 0 / Para arriba: -1

In [102]:
list_of_voids = search_voids_bb_neightbours(df_predictions= df_predictions, dict_of_neightbours= dict_of_neightbours[0])

Evaluating 572...
Evaluating 1059...
Evaluating 581...
Evaluating 585...
Evaluating 1203...
Evaluating 576...
Evaluating 432...
Evaluating 1067...
Evaluating 577...
Evaluating 583...
Evaluating 574...
Evaluating 1172...
Evaluating 580...
Evaluating 1205...
Evaluating 1054...
Evaluating 1297...
Evaluating 496...
Evaluating 571...
Evaluating 494...
Evaluating 586...
Evaluating 1294...
Evaluating 431...
Evaluating 1235...
Evaluating 1237...
Evaluating 834...
Evaluating 89...
Evaluating 1060...
Evaluating 1201...
Evaluating 459...
Evaluating 425...
Evaluating 457...
Evaluating 579...
Evaluating 564...
Evaluating 696...
Evaluating 825...
Evaluating 393...
Evaluating 430...
Evaluating 230...
Evaluating 1174...
Evaluating 399...
Evaluating 66...
Evaluating 391...
Evaluating 587...
Evaluating 273...
Evaluating 489...
Evaluating 1053...
Evaluating 561...
Evaluating 143...
Evaluating 422...
Evaluating 225...
Evaluating 277...
Evaluating 185...
Evaluating 270...
Evaluating 1132...
Evaluating 1260

In [11]:
img_path = '/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg'

In [14]:
list_of_dicts = get_neightbours(img_path= img_path, df_predictions= df_predictions)

In [24]:
df_predictions.loc[572]

X_center    221.571442
Y_center    307.081207
Width        23.514479
Height       68.048775
Name: 572, dtype: float64

In [36]:
def get_neightbours_test(img_path:str, df_predictions:pd.DataFrame, neightbour:str=('left','right','up')) -> list:

    dict_l_n = {} # neigh left
    dict_r_n = {} # neigh right
    dict_h_n= {}  # alone right

    for index_bb in df_predictions.index:
        print(index_bb)
        list_of_neightbours_l = []
        list_of_neightbours_r = []
        list_of_neightbours_h = [] 
        list_of_alones_l = []
        list_of_alones_r = []
        list_of_alones_h=[]
        for index in df_predictions.index:
            print(index)

            X_center_0 = df_predictions.loc[index_bb][0]
            Y_center_0 = df_predictions.loc[index_bb][1]
            Width_0 = df_predictions.loc[index_bb][2]
            height_0 = df_predictions.loc[index_bb][3]
            threshold_x_0 = 0.8 * Width_0
            threshold_y_0 = 0.8 * height_0

            X_center_neightbour = df_predictions.loc[index][0]
            Y_center_neightbour = df_predictions.loc[index][1]
            a=4
            k=1.2

        if neightbour == 'left':
            ### Neightbor Left
            x_min_l = X_center_0 - Width_0 - (a*threshold_x_0)
            x_max_l = X_center_0 - Width_0 + (k*threshold_x_0)

            y_min_l = Y_center_0 - (k*threshold_y_0)
            y_max_l = Y_center_0 + (k*threshold_y_0)

        elif neightbour == 'right': 
            ### Neightbor Right
            x_min_r = X_center_0 + Width_0 - (k*threshold_x_0)
            x_max_r = X_center_0 + Width_0 + (a*threshold_x_0)

            y_min_r = Y_center_0 - (k*threshold_y_0)
            y_max_r = Y_center_0 + (k*threshold_y_0)

        else:
            ### Neightbor High
            x_min_h = X_center_0 - (k*threshold_x_0)
            x_max_h = X_center_0 + (2*k*threshold_x_0)

            y_min_h = Y_center_0 + height_0 - (0.8*threshold_y_0)
            y_max_h = Y_center_0 + height_0 + (2*a*threshold_y_0)
            
            ### Search neig left
            if (x_min_l < X_center_neightbour < x_max_l and y_min_l < Y_center_neightbour < y_max_l )  :
                list_of_neightbours_l.append([index])
                try:
                    dict_l_n[str(index_bb)].append(index)
                except KeyError:
                    dict_l_n[str(index_bb)]=[]
                    dict_l_n[str(index_bb)].append(index)
                
            else:
                list_of_alones_l.append([index])
            if (x_min_r < X_center_neightbour < x_max_r and y_min_r < Y_center_neightbour < y_max_r ):
                list_of_neightbours_r.append([index])
                try:
                    dict_r_n[str(index_bb)].append(index)
                except KeyError:
                    dict_r_n[str(index_bb)]=[]
                    dict_r_n[str(index_bb)].append(index)
            else:

                list_of_alones_r.append([index_bb, index])
                
            # Search neig hight    
            if (x_min_h < X_center_neightbour < x_max_h and y_min_h < Y_center_neightbour < y_max_h ):
                list_of_neightbours_h.append([index])
                try:
                    dict_h_n[str(index_bb)].append(index)
                except KeyError:
                    dict_h_n[str(index_bb)]=[]
                    dict_h_n[str(index_bb)].append(index)
            else:
                list_of_alones_h.append([index_bb, index])
    print('listo primer loop')

    return [dict_l_n, dict_r_n, dict_h_n]

In [ ]:
for i in df_predictions.index:
    print(type(i))

In [57]:
df_predictions.loc[572].values

array([221.57144165, 307.08120728,  23.51447868,  68.04877472])

In [76]:
df_predictions.loc[df_predictions.index == 572].values

array([[221.57144165, 307.08120728,  23.51447868,  68.04877472]])

In [241]:
X_center_0 = df_predictions.loc[572][0]
Y_center_0 = df_predictions.loc[572][1]
INPUT_WH_YOLO = 640

In [232]:
X_center_0 + 160

381.5714416503906

In [221]:
filter_1 = (df_predictions.loc[:, 'X_center'] > (X_center_0 - INPUT_WH_YOLO/4))

In [222]:
filter_2 = ((df_predictions.loc[:, 'X_center'] < (X_center_0 + INPUT_WH_YOLO/4)))

In [223]:
df_predictions_pre = df_predictions[filter_1]

In [225]:
df_predictions_last =  df_predictions[filter_2]

In [242]:
df_predictions_final = df_predictions[(df_predictions.loc[:, 'X_center'] < (X_center_0 + INPUT_WH_YOLO/8)) & (df_predictions.loc[:, 'X_center'] > (X_center_0 - INPUT_WH_YOLO/8)) & (df_predictions.loc[:, 'Y_center'] > (Y_center_0 - INPUT_WH_YOLO/8)) & (df_predictions.loc[:, 'Y_center'] < (Y_center_0 + INPUT_WH_YOLO/8))]

In [245]:
filter_1 = (df_predictions.loc[:, 'X_center'] < (X_center_0 + INPUT_WH_YOLO/8))
filter_2 = (df_predictions.loc[:, 'X_center'] > (X_center_0 - INPUT_WH_YOLO/8))
filter_3 = (df_predictions.loc[:, 'Y_center'] > (Y_center_0 - INPUT_WH_YOLO/8))
filter_4 = (df_predictions.loc[:, 'Y_center'] < (Y_center_0 + INPUT_WH_YOLO/8))
filter_final = (filter_1 & filter_2) & (filter_3 & filter_4)

In [248]:
df_predictions_final_2 = df_predictions[filter_final]

In [249]:
len(df_predictions_final_2)

20

In [42]:
def get_neightbours_test(df_predictions:pd.DataFrame, neightbour:str, index_it:int) -> dict:

    dict_of_neightbours = {} 

    X_center_0 = df_predictions.loc[index_it][0]
    Y_center_0 = df_predictions.loc[index_it][1]
    Width_0 = df_predictions.loc[index_it][2]
    height_0 = df_predictions.loc[index_it][3]
    threshold_x_0 = 0.8 * Width_0
    threshold_y_0 = 0.8 * height_0

    INPUT_WH_YOLO = 640
    filter_1 = (df_predictions.loc[:, 'X_center'] < (X_center_0 + INPUT_WH_YOLO/8))
    filter_2 = (df_predictions.loc[:, 'X_center'] > (X_center_0 - INPUT_WH_YOLO/8))
    filter_3 = (df_predictions.loc[:, 'Y_center'] > (Y_center_0 - INPUT_WH_YOLO/8))
    filter_4 = (df_predictions.loc[:, 'Y_center'] < (Y_center_0 + INPUT_WH_YOLO/8))
    filter_final = (filter_1 & filter_2) & (filter_3 & filter_4)

    df_predictions_final_2 = df_predictions[filter_final]

    for index_bb in df_predictions_final_2.index:

        list_of_neightbours_l = []
        list_of_neightbours_r = []
        list_of_neightbours_h = [] 
        list_of_alones_l = []
        list_of_alones_r = []
        list_of_alones_h=[]

        X_center_neightbour = df_predictions.loc[index_bb][0]
        Y_center_neightbour = df_predictions.loc[index_bb][1]
        a = 4
        k = 1.2

        ### Neightbor Left
        if neightbour == 'left':
            
            x_min_l = X_center_0 - Width_0 - (a*threshold_x_0)
            x_max_l = X_center_0 - Width_0 + (k*threshold_x_0)

            y_min_l = Y_center_0 - (k*threshold_y_0)
            y_max_l = Y_center_0 + (k*threshold_y_0)

            if (x_min_l < X_center_neightbour < x_max_l and y_min_l < Y_center_neightbour < y_max_l )  :
                list_of_neightbours_l.append([index_bb])
                try:
                    dict_of_neightbours[str(index_it)].append(index_bb)
                except KeyError:
                    dict_of_neightbours[str(index_it)]= []
                    dict_of_neightbours[str(index_it)].append(index_bb)
            else:
                list_of_alones_l.append([index_bb])

        ### Neightbor Right
        elif neightbour == 'right':

            x_min_r = X_center_0 + Width_0 - (k*threshold_x_0)
            x_max_r = X_center_0 + Width_0 + (a*threshold_x_0)

            y_min_r = Y_center_0 - (k*threshold_y_0)
            y_max_r = Y_center_0 + (k*threshold_y_0)

            if (x_min_r < X_center_neightbour < x_max_r and y_min_r < Y_center_neightbour < y_max_r ):
                list_of_neightbours_r.append([index_bb])
                try:
                    dict_of_neightbours[str(index_it)].append(index_bb)
                except KeyError:
                    dict_of_neightbours[str(index_it)]= []
                    dict_of_neightbours[str(index_it)].append(index_bb)
            else:
                list_of_alones_r.append([index_it, index_bb])

        ### Neightbor High
        else:
            x_min_h = X_center_0 - (k*threshold_x_0)
            x_max_h = X_center_0 + (2*k*threshold_x_0)

            y_min_h = Y_center_0 + height_0 - (0.8*threshold_y_0)
            y_max_h = Y_center_0 + height_0 + (2*a*threshold_y_0)

            if (x_min_h < X_center_neightbour < x_max_h and y_min_h < Y_center_neightbour < y_max_h ):
                list_of_neightbours_h.append([index_bb])
                try:
                    dict_of_neightbours[str(index_it)].append(index_bb)
                except KeyError:
                    dict_of_neightbours[str(index_it)]= []
                    dict_of_neightbours[str(index_it)].append(index_bb)
            else:
                list_of_alones_h.append([index_it, index_bb])

    return dict_of_neightbours

In [43]:
pool = multiprocessing.Pool()
neightbour = 'left'
func = partial(get_neightbours_test, df_predictions, neightbour)
dict_of_neightbours_left = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_left = list(filter(None, dict_of_neightbours_left))

pool = multiprocessing.Pool()
neightbour = 'right'
func = partial(get_neightbours_test, df_predictions, neightbour)
dict_of_neightbours_right = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_right = list(filter(None, dict_of_neightbours_right))

pool = multiprocessing.Pool()
neightbour = 'up'
func = partial(get_neightbours_test, df_predictions, neightbour)
dict_of_neightbours_up = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_up = list(filter(None, dict_of_neightbours_up))

In [45]:
dict_of_neightbours_left

[{'572': [571, 273, 277, 275, 599, 600]},
 {'1059': [1054, 825, 1055]},
 {'581': [576, 577, 580, 579, 317, 20]},
 {'585': [581, 583, 580, 579, 20, 612]},
 {'1203': [1201, 430, 166, 428]},
 {'576': [572, 574, 571, 277]},
 {'432': [431, 430, 1145]},
 {'1067': [581, 577, 583, 580, 1297, 1294, 579, 1296]},
 {'577': [572, 576, 574, 571]},
 {'583': [581, 577, 580, 579, 317, 20]},
 {'574': [572, 571, 277, 275]},
 {'1172': [432, 431]},
 {'580': [576, 577, 574, 579, 317]},
 {'1205': [1203, 1201]},
 {'1054': [1053, 234]},
 {'1297': [1294, 1060, 1296]},
 {'496': [494, 1237]},
 {'571': [564, 273, 277, 270, 275, 562, 599, 600]},
 {'494': [1235, 1237]},
 {'586': [581, 585, 583, 580, 20, 612]},
 {'1294': [572, 1059, 576, 574, 571, 1060, 825, 277]},
 {'431': [430, 428, 1145]},
 {'1235': [1233]},
 {'1237': [1235, 1233]},
 {'834': [1067, 1297, 1296]},
 {'89': [1152, 99]},
 {'1060': [1059, 825, 277, 275, 1055]},
 {'1201': [166, 1198]},
 {'459': [1205, 457]},
 {'425': [422, 420, 120]},
 {'457': [1203, 120

In [66]:
h_image, w_image = image.shape[0:2]

In [64]:
type(h_image)

int

In [65]:
def search_voids_bb_neightbours(df_predictions: pd.DataFrame, dict_of_neightbours: dict, h_image:int, w_image:int, neightbour:str=('left','right','up')):
    
    list_of_voids = []

    k = 0
    z = 0
    if neightbour == 'left':
        k = -1
        z = 0
    elif neightbour == 'right':
        k = 1
        z = 0
    else:
        k = 0
        z = -1

    # iterate over all dicts in list
    for dic in dict_of_neightbours:

        # iterate over key and value pairs of dict (index_a = key // index_b = value pair)
        for index_a, index_b in dic.items():

            #h_image, w_image = image.shape[0:2] # limits of the image
            w_index_a = df_predictions.loc[int(index_a)][2]
            h_index_a = df_predictions.loc[int(index_a)][3]

            
            # Virtual bounding box to evaluate neightbours 
            xA1 = df_predictions.loc[int(index_a)][0] - df_predictions.loc[int(index_a)][2]/2 + (k * w_index_a) # Para izquierda: (-1) / Para derecha: 1 / Para arriba: 0
            yA1 = df_predictions.loc[int(index_a)][1] - df_predictions.loc[int(index_a)][3]/2 + (z * h_index_a) # Para izquierda: 0 / Para derecha: 0 / Para arriba: (-1)
            xA2 = df_predictions.loc[int(index_a)][0] + df_predictions.loc[int(index_a)][2]/2 + (k * w_index_a) # Para izquierda: (-1) / Para derecha: 1 / Para arriba: 0
            yA2 = df_predictions.loc[int(index_a)][1] + df_predictions.loc[int(index_a)][3]/2 + (z * h_index_a) # Para izquierda: 0 / Para derecha: 0 / Para arriba: -1
            boxA = [xA1, yA1, xA2, yA2]


            X_center_A = df_predictions.loc[int(index_a)][0] - k * df_predictions.loc[int(index_a)][2]  # Left X_center - Width  // Right X_center + Width
            Y_center_A = df_predictions.loc[int(index_a)][1]  - k * df_predictions.loc[int(index_a)][2]  # Left Y_center - Width // Right Y_center + Width

            print(f'Evaluating {index_a}...')
            

            # Limits of the image
            if 0 < xA1 < w_image and 0 < xA2 < w_image and 0 < yA1 < h_image and  0 < yA2 < h_image:
                    trigger = True
                    void_number = 0

                    # Iterate over each neightbour: neightbour vs virtual bounding box
                    for item in index_b:
                        
                        first_list = []
                        xB1 = df_predictions.loc[item][0] - df_predictions.loc[item][2]/2
                        yB1 = df_predictions.loc[item][1] - df_predictions.loc[item][3]/2
                        xB2 = df_predictions.loc[item][0] + df_predictions.loc[item][2]/2
                        yB2 = df_predictions.loc[item][1] + df_predictions.loc[item][3]/2
                        boxB = [xB1, yB1,xB2, yB2]

                        xA = max(boxA[0], boxB[0])
                        yA = max(boxA[1], boxB[1])
                        xB = min(boxA[2], boxB[2])
                        yB = min(boxA[3], boxB[3])

                        interArea = (xB - xA) * (yB - yA)

                        boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
                        boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

                        iou = interArea / float(boxAArea + boxBArea - interArea)
                        trigger = trigger and (iou < 0.1)
                    #print(f'Trigger: {trigger}')

                    if trigger == False:
                        pass
                        #print(f'No hay espacio vacio a la izquierda de {index_a}')
                    else:
                        print(f'A la izquierda de {index_a} hay espacio vacio')
                        void_number += 1
                        void_text = f'{neightbour} void #{void_number}'
                        first_list.append(index_a)
                        first_list.append(void_text)
                        first_list.append(X_center_A)
                        first_list.append(Y_center_A)
                        list_of_voids.append(first_list)
                    

    return list_of_voids

In [67]:
list_of_voids_left = search_voids_bb_neightbours(df_predictions=df_predictions, dict_of_neightbours=dict_of_neightbours_left, h_image=h_image, w_image=w_image, neightbour='left')

Evaluating 572...
Evaluating 1059...
Evaluating 581...
Evaluating 585...
Evaluating 1203...
Evaluating 576...
Evaluating 432...
Evaluating 1067...
Evaluating 577...
Evaluating 583...
Evaluating 574...
Evaluating 1172...
Evaluating 580...
Evaluating 1205...
Evaluating 1054...
Evaluating 1297...
Evaluating 496...
Evaluating 571...
Evaluating 494...
Evaluating 586...
Evaluating 1294...
Evaluating 431...
Evaluating 1235...
Evaluating 1237...
Evaluating 834...
Evaluating 89...
Evaluating 1060...
Evaluating 1201...
Evaluating 459...
Evaluating 425...
Evaluating 457...
Evaluating 579...
Evaluating 564...
Evaluating 696...
Evaluating 825...
Evaluating 393...
Evaluating 430...
Evaluating 230...
Evaluating 1174...
Evaluating 399...
Evaluating 66...
Evaluating 391...
Evaluating 587...
Evaluating 273...
Evaluating 489...
Evaluating 1053...
Evaluating 561...
Evaluating 143...
Evaluating 422...
Evaluating 225...
Evaluating 277...
Evaluating 185...
Evaluating 270...
Evaluating 1132...
Evaluating 1260

In [68]:
list_of_voids_left

[['1233', 'left void #1', 106.61637687683105, 573.5148296356201],
 ['234', 'left void #1', 111.32217407226562, 258.05506896972656],
 ['92', 'left void #1', 437.14215660095215, 471.32013511657715],
 ['791', 'left void #1', 99.01019668579102, 160.16558456420898],
 ['59', 'left void #1', 458.86724281311035, 427.8168888092041],
 ['533', 'left void #1', 475.07324409484863, 162.59886360168457],
 ['23', 'left void #1', 470.9087085723877, 381.87278938293457],
 ['310', 'left void #1', 437.86330699920654, 328.39715099334717],
 ['43', 'left void #1', 474.26671028137207, 405.9255847930908]]